## 1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import json
from IR.Searcher.Index_Searcher import Index_Searcher
from Fine_Tuning.Generator.Recommendations_Keyphrase import Recommmender

## 2. Load Test data

In [2]:
file_path = '../Data/All_Data.json'

In [3]:
with open('../Data/Augmented/Train_Test/test.json', 'r') as file:
    data = file.read()
    data = json.loads(data)

test_df = pd.DataFrame.from_dict(data)

In [4]:
test_df.head()

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description,effective_queries
0,289455,[org.eclipse.jdt.junit/src/org/eclipse/jdt/int...,eclipse.jdt.ui,[start create JUnit active active Bug active s...,[JUnit] JUnit view not active after test,I20090915-. JUnit view is not active after tes...,[start create JUnit active active Bug active s...
1,165045,[ui/org.eclipse.pde.core/src/org/eclipse/pde/i...,eclipse.pde.ui,[icons actions importing created project build...,Import plugin with source leaves a warning ab...,After exporting a plug-in project with source ...,[icons actions importing created project build...
2,49030,[java/org/apache/catalina/core/StandardService...,tomcat70,[listening situation uninitialized xml Server ...,Processing of Tomcat Connector configuration ...,If the Server part of the Tomcat configuratio...,[listening situation uninitialized xml Server ...
3,55582,[java/org/apache/jasper/compiler/TagFileProces...,tomcat70,[threads lines File TagFileProcessor wrapperUr...,Concurrent issue of TagFileProcessor,Created attachment [details] Unsynchronized g...,[threads lines File TagFileProcessor wrapperUr...
4,134483,[framework/bundles/org.eclipse.ecf/src/org/ecl...,ecf,[Application Container Standalone Factory Cont...,Standalone ClientApplication is breaks in lin...,The standalone org.eclipse.ecf.provider.app.Cl...,[Application Container Standalone Factory Cont...


## Iterate over test data and get query recommendations

In [8]:
model_name = "ml6team/keyphrase-generation-t5-small-inspec"
model_path = '../Models_Fine_Tuned/keyphrase-generation-t5-small-inspec-20230806_0305-50ep'
# create instance of Recommender
recommender = Recommmender(model_name=model_name, model_path=model_path)

In [9]:
searcher = Index_Searcher()

In [12]:
# create column 'query_recommendations' in test_df of type list of lists
test_df['query_recommendations'] = test_df['bug_description'].apply(lambda x: [])

In [14]:
test_df.shape

(100, 8)

In [15]:
# iterate over test_df
for index, row in test_df.iterrows():
    # get query
    bug_description = row['bug_description']

    # get query recommendations: list
    query_recommendations = recommender.get_recommendations(bug_description=bug_description, num_of_recommendations=5)

    # print(query_recommendations)
    # break

    # save query recommendations in test_df as list in column 'query_recommendations'
    test_df.at[index, 'query_recommendations'] = query_recommendations

    # search_results_collection = []
    #
    # # iterate over query recommendations
    # for recommendation in query_recommendations:
    #     query = ' '.join(recommendation)
    #
    #     # get search results
    #     search_results = searcher.search(query=query, num_of_results=10)
    #
    #     # append search results to search_results_collection
    #     search_results_collection.append(search_results)
    #
    # # Majority Voting Scorer
    # # Scoring: after each round of searches for recommended queries, score the results
    # majority_scorer = MajorityVotingScorer(List_Collection=search_results_collection)
    # scored_list = majority_scorer.score_items(do_sort=True, only_keys_array=True)




In [16]:
test_df.head()

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description,effective_queries,query_recommendations
0,289455,[org.eclipse.jdt.junit/src/org/eclipse/jdt/int...,eclipse.jdt.ui,[start create JUnit active active Bug active s...,[JUnit] JUnit view not active after test,I20090915-. JUnit view is not active after tes...,[start create JUnit active active Bug active s...,"[[JUnit, Java, fresh, Bug, view, Explorer], [J..."
1,165045,[ui/org.eclipse.pde.core/src/org/eclipse/pde/i...,eclipse.pde.ui,[icons actions importing created project build...,Import plugin with source leaves a warning ab...,After exporting a plug-in project with source ...,[icons actions importing created project build...,"[[export, file, references, Export, files, exp..."
2,49030,[java/org/apache/catalina/core/StandardService...,tomcat70,[listening situation uninitialized xml Server ...,Processing of Tomcat Connector configuration ...,If the Server part of the Tomcat configuratio...,[listening situation uninitialized xml Server ...,"[[problem, problem, wrong, wrong, incorrect, c..."
3,55582,[java/org/apache/jasper/compiler/TagFileProces...,tomcat70,[threads lines File TagFileProcessor wrapperUr...,Concurrent issue of TagFileProcessor,Created attachment [details] Unsynchronized g...,[threads lines File TagFileProcessor wrapperUr...,"[[issue, instance, Thread, Unsynchronized, Jsp..."
4,134483,[framework/bundles/org.eclipse.ecf/src/org/ecl...,ecf,[Application Container Standalone Factory Cont...,Standalone ClientApplication is breaks in lin...,The standalone org.eclipse.ecf.provider.app.Cl...,[Application Container Standalone Factory Cont...,"[[Eclipse, ExceptionInInitializerError, Client..."


In [17]:
# save the test_df as json
test_df.to_json('../Data/Outputs/test_recommendations.json', orient='records')

In [18]:
# # load the test_df from json
# with open('../Data/Outputs/test_recommendations.json', 'r') as file:
#     data = file.read()
#     data = json.loads(data)
#
# test_df = pd.DataFrame.from_dict(data)

In [19]:
# test_df.head()

,bug_id,ground_truth,repo,reformed_query,bug_title,bug_description,effective_queries,query_recommendations
0,289455,[org.eclipse.jdt.junit/src/org/eclipse/jdt/int...,eclipse.jdt.ui,[start create JUnit active active Bug active s...,[JUnit] JUnit view not active after test,I20090915-. JUnit view is not active after tes...,[start create JUnit active active Bug active s...,"[[JUnit, Java, fresh, Bug, view, Explorer], [J..."
1,165045,[ui/org.eclipse.pde.core/src/org/eclipse/pde/i...,eclipse.pde.ui,[icons actions importing created project build...,Import plugin with source leaves a warning ab...,After exporting a plug-in project with source ...,[icons actions importing created project build...,"[[export, file, references, Export, files, exp..."
2,49030,[java/org/apache/catalina/core/StandardService...,tomcat70,[listening situation uninitialized xml Server ...,Processing of Tomcat Connector configuration ...,If the Server part of the Tomcat configuratio...,[listening situation uninitialized xml Server ...,"[[problem, problem, wrong, wrong, incorrect, c..."
3,55582,[java/org/apache/jasper/compiler/TagFileProces...,tomcat70,[threads lines File TagFileProcessor wrapperUr...,Concurrent issue of TagFileProcessor,Created attachment [details] Unsynchronized g...,[threads lines File TagFileProcessor wrapperUr...,"[[issue, instance, Thread, Unsynchronized, Jsp..."
4,134483,[framework/bundles/org.eclipse.ecf/src/org/ecl...,ecf,[Application Container Standalone Factory Cont...,Standalone ClientApplication is breaks in lin...,The standalone org.eclipse.ecf.provider.app.Cl...,[Application Container Standalone Factory Cont...,"[[Eclipse, ExceptionInInitializerError, Client..."
